<a href="https://colab.research.google.com/github/aks-vijay/Apache-Spark/blob/main/Ingest%2C_Wrangle_%26_Export_Races_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 609.7 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58749 sha256=8fb51b7fe1208aa9a2475db4693e73ddd59d2e9ac3a64a3fbefe93d0d0d8bda1
  Stored in directory: /root/.cache/pip/wheels/63/88/77/b4131110ea4094540f7b47c6d62a649807d7e94800da5eab0b
Successfully built spark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=b023574d30c29316785a5292c98e2f529f02af36a041a7916f320f2fb518fc15
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession  \
          .builder \
          .appName("MyLocalSparkApp") \
          .master("local[*]") \
          .getOrCreate()

In [3]:
from google.colab import files
uploaded = files.upload()

Saving circuits.csv to circuits.csv


In [4]:
from google.colab import files
uploaded = files.upload()

Saving races.csv to races.csv


# Ingest data to Spark Dataframes

In [9]:
# load the file
from pyspark.sql.types import *

circuits_user_defined_schema = StructType(fields=
                                 [StructField('circuitId', IntegerType(), True),
                                  StructField('circuitRef', StringType(), True),
                                  StructField('name', StringType(), True),
                                  StructField('location', StringType(), True),
                                  StructField('country', StringType(), True),
                                  StructField('lat', DoubleType(), True),
                                  StructField('lng', DoubleType(), True),
                                  StructField('alt', IntegerType(), True),
                                  StructField('url', StringType(), True)
                                  ])
df_circuits = spark.read \
  .schema(schema=circuits_user_defined_schema) \
  .option("header", True) \
  .csv('circuits.csv')

races_user_defined_schema = StructType(fields=
                                        [StructField('raceId', IntegerType(), True),
                                          StructField('year', IntegerType(), True),
                                          StructField('round', IntegerType(), True),
                                          StructField('circuitId', IntegerType(), True),
                                          StructField('name', StringType(), True),
                                          StructField('date', DateType(), True),
                                          StructField('time', StringType(), True),
                                          StructField('url', StringType(), True)]
                                       )
df_races = spark.read \
            .schema(schema=races_user_defined_schema) \
            .option("header", True) \
            .csv("races.csv")

# Check for any duplicates

In [11]:
# check for duplicates
df_circuits.exceptAll(df_circuits.dropDuplicates()).show()
df_races.exceptAll(df_races.dropDuplicates()).show()

+---------+----------+----+--------+-------+---+---+---+---+
|circuitId|circuitRef|name|location|country|lat|lng|alt|url|
+---------+----------+----+--------+-------+---+---+---+---+
+---------+----------+----+--------+-------+---+---+---+---+

+------+----+-----+---------+----+----+----+---+
|raceId|year|round|circuitId|name|date|time|url|
+------+----+-----+---------+----+----+----+---+
+------+----+-----+---------+----+----+----+---+



# Create Lookup Table for Country Codes Mapping

In [39]:
# create mapping for country codes

country_codes = {
    "Russia": "RU",
    "Sweden": "SE",
    "Malaysia": "MY",
    "Singapore": "SG",
    "Turkey": "TR",
    "Germany": "DE",
    "France": "FR",
    "Argentina": "AR",
    "Belgium": "BE",
    "China": "CN",
    "India": "IN",
    "Italy": "IT",
    "Spain": "ES",
    "Monaco": "MC",
    "Morocco": "MA",
    "USA": "US",
    "Mexico": "MX",
    "Azerbaijan": "AZ",
    "UK": "UK",
    "Saudi Arabia": "SA"
}

country_codes_lists = list(country_codes.items())
country_codes_lookup = spark.createDataFrame(country_codes_lists, schema=["country", "country_codes"])

# Clean the dataframes for Reporting

In [61]:
df_races_cleaned = df_races \
                    .withColumnRenamed("raceId", "race_id") \
                    .withColumnRenamed("circuitId", "circuit_id") \
                    .withColumn("race_date_time", concat(col("date"), lit(" "), col("time"))) \
                    .drop("date", col("time")) \
                    .withColumn("ingestion_date", current_timestamp())

columns_to_select = ["circuit_id", "circuit_ref", "name", "location", "country", "location_and_country", "lat", "lng", "alt", "url"]

df_circuits_cleaned = df_circuits \
                        .withColumnRenamed("circuitId", "circuit_id") \
                        .withColumnRenamed("circuitRef", "circuit_ref") \
                        .withColumn("location_and_country", concat(col("location"), lit(", "), col("country"))) \
                        .join(country_codes_lookup, df_circuits.country==country_codes_lookup.country, "inner") \
                        .drop(df_circuits["country"], country_codes_lookup["country"]) \
                        .withColumnRenamed("country_codes", "country") \
                        .select(*columns_to_select)


df_races_circuits_cleaned = df_races_cleaned \
                              .join(df_circuits_cleaned, df_races_cleaned.circuit_id == df_circuits_cleaned.circuit_id, "inner") \
                              .select(df_races_cleaned.race_id,
                                      df_races_cleaned.year.alias("race_year"),
                                      df_races_cleaned.round,
                                      df_races_cleaned.circuit_id,
                                      df_races_cleaned.name.alias("race_name"),
                                      df_races_cleaned.race_date_time,
                                      df_circuits_cleaned.name.alias("circuit_name"),
                                      df_circuits_cleaned.location.alias("circuit_location"),
                                      df_circuits_cleaned.country.alias("circuit_country"),
                                      df_circuits_cleaned.lat.alias("latitude"),
                                      df_circuits_cleaned.lng.alias("longitude"),
                                      df_races_cleaned.url.alias("race_url"),
                                      df_circuits_cleaned.url.alias("circuit_url"),
                                      df_races_cleaned.ingestion_date)

# Functions to Answer business questions for Analytics teams

In [ ]:
# function to answer business questions
# Total races occured per Country by Year
def total_races_per_country(year) -> DataFrame:
  df_races_circuits_cleaned \
      .where(col("race_year") == year) \
      .groupBy("circuit_country") \
      .agg(
          count("*").alias("total_races_per_country")
      ) \
      .orderBy("total_races_per_country", ascending=False) \
      .show()

def select_race_by_country(country_code, limit_value=5) -> DataFrame:
  df_races_circuits_cleaned \
    .where(col("circuit_country") == country_code) \
    .show(limit_value, truncate=False)

def races_count_by_country(country) -> int:
  count = df_races_circuits_cleaned \
            .where(col("circuit_country") == country) \
            .count()

  return count

# total_races_per_country(2021)
# select_race_by_country("UK")
# races_count_by_country("US")

# Write the cleaned dataframe for Downstream Reporting

In [104]:
df_races_circuits_cleaned.write \
  .mode("overwrite") \
  .csv("races_by_circuits_cleaned")

In [74]:
spark.stop()